In [1]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
import numpy as np
from skimage import color, io
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

In [2]:
tf.set_random_seed(0)
batch_size = 1

In [19]:
def convertToLab(image, label):
    lab = color.rgb2lab(label)
    X_batch = lab[:,:,0]
    Y_batch = lab[:,:,1:]
    return X_batch.reshape(X_batch.shape+(1,)), Y_batch


def _parse_function(filename):
    image = tf.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [256, 256], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    label = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.divide(label, 255.0)
    
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    return image, label

file_paths = map((lambda x: "../data/" + x), os.listdir("../data/"))

dataset = tf.data.Dataset.from_tensor_slices(file_paths)
dataset = dataset.map(_parse_function)
dataset = dataset.map(lambda image, label: 
    tuple(tf.py_func(
        convertToLab, [image, label], [tf.double, tf.double]
    ))
)
dataset = dataset.batch(batch_size)

iterator = dataset.make_one_shot_iterator()

In [20]:
#file_paths = map((lambda x: "../data/" + x), os.listdir("../data/"))
#
#all_images = ops.convert_to_tensor(file_paths, dtype=dtypes.string)
#
#train_input_queue = tf.train.slice_input_producer(
#                                    [all_images, all_images],
#                                    shuffle=False)
#
#
#file_content = tf.read_file(train_input_queue[0])
#train_image = tf.image.decode_jpeg(file_content, channels=3)
#train_image = tf.image.resize_images(train_image, [256, 256], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#train_image = tf.image.rgb_to_grayscale(train_image)
#train_image = tf.image.convert_image_dtype(train_image, tf.float32)
#
#label_content = tf.read_file(train_input_queue[1])
#train_label = tf.image.decode_jpeg(file_content, channels=3)
#train_label = tf.image.resize_images(train_label, [256, 256], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#train_label = tf.image.convert_image_dtype(train_label, tf.float32)
#train_label = tf.divide(train_label, 255.0)
#
#
#                    
## collect batches of images before processing
#train_image_batch, train_label_batch = tf.train.batch(
#    [train_image, train_label],
#    batch_size=batch_size
#    #,num_threads=1
#)

In [21]:
def plotImages(num, imgs):
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(num): #length of your filename list
        image = imgs.eval() 
        plt.imshow(image)
        plt.show()

    coord.request_stop()
    coord.join(threads)

In [22]:
def prepareImages(imgs):
    sz_imgs = tf.image.resize_images(imgs, [256, 256], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    bw_imgs = tf.image.rgb_to_grayscale(sz_imgs)
    
    return sz_imgs, bw_imgs

In [23]:
def imageLabGenerator(size):
    for batch in datagen.flow(Xtrain, batch_size=size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [24]:
def conv2DRelu(X, W, B, strides, padding):
    # strides: [batch_step, height_step, width_step, channel_step] 
    return tf.nn.relu(tf.nn.conv2d(X, W, strides=strides, padding=padding) + B)

def conv2DTanh(X, W, B, strides, padding):
    # strides: [batch_step, height_step, width_step, channel_step] 
    return tf.nn.tanh(tf.nn.conv2d(X, W, strides=strides, padding=padding) + B)

def weight(width, height, input_channels, output_channels):
    # [width, height, input channel, output channel]
    return tf.Variable(tf.truncated_normal([width, height, input_channels, output_channels], stddev=0.1))

def bias(outputChannels):
    return tf.Variable(tf.zeros([outputChannels])) # bias for each output channel.

def upSampling2D(X, height, width):
    return tf.image.resize_images(X, [height, width], tf.image.ResizeMethod.NEAREST_NEIGHBOR)

In [25]:
# The input images x will consist of a 2d tensor of floating point numbers.
#X = tf.placeholder(tf.float32, shape=[None, [256, 256, 1]]) # 256 x 256
# The target output y_ will also consist of a 2d tensor, where each row is 
# a one-hot 2-dimensional vector indicating a b values.
Y_ = tf.placeholder(tf.float32, shape=[None, 256, 256, 2])

# Input Layer
X = tf.placeholder(tf.float32, shape=[None, 256, 256, 1]) # 256 x 256

# Conv layer 1 
W1 = weight(5, 5, 1, 64)
B1 = bias(64)
Y1 = conv2DRelu(X, W1, B1, [1,1,1,1], 'SAME')

# Conv layer 2
W2 = weight(5, 5, 64, 64)
B2 = bias(64)
Y2 = conv2DRelu(Y1, W2, B2, [1, 2, 2, 1], 'SAME')

# Conv layer 3
W3 = weight(5, 5, 64, 128)
B3 = bias(128)
Y3 = conv2DRelu(Y2, W3, B3, [1, 1, 1, 1], 'SAME')

# Conv layer 4
W4 = weight(5, 5, 128, 128)
B4 = bias(128)
Y4 = conv2DRelu(Y3, W4, B4, [1, 2, 2, 1], 'SAME')

# Conv layer 5
W5 = weight(5, 5, 128, 256)
B5 = bias(256)
Y5 = conv2DRelu(Y4, W5, B5, [1, 1, 1, 1], 'SAME')

# Conv layer 6
W6 = weight(5, 5, 256, 256)
B6 = bias(256)
Y6 = conv2DRelu(Y5, W6, B6, [1, 2, 2, 1], 'SAME')

# Conv layer 7
W7 = weight(5, 5, 256, 512)
B7 = bias(512)
Y7 = conv2DRelu(Y6, W7, B7, [1, 1, 1, 1], 'SAME')

# Conv layer 8
W8 = weight(5, 5, 512, 256)
B8 = bias(256)
Y8 = conv2DRelu(Y7, W8, B8, [1, 1, 1, 1], 'SAME')

# Conv layer 9
W9 = weight(5, 5, 256, 128)
B9 = bias(128)
Y9 = conv2DRelu(Y8, W9, B9, [1, 1, 1, 1], 'SAME')

# Up sampling layer 1
Y10 = upSampling2D(Y9, 64, 64)

# Conv layer 10
W11 = weight(5, 5, 128, 64)
B11 = bias(64)
Y11 = conv2DRelu(Y10, W11, B11, [1, 1, 1, 1], 'SAME')

# Up sampling layer 2
Y12 = upSampling2D(Y11, 128, 128)

# Conv layer 11
W13 = weight(5, 5, 64, 32)
B13 = bias(32)
Y13 = conv2DRelu(Y12, W13, B13, [1, 1, 1, 1], 'SAME')

# Conv layer 12
W14 = weight(5, 5, 32, 2)
B14 = bias(2)
Y14 = conv2DTanh(Y13, W14, B14, [1, 1, 1, 1], 'SAME')

# Up sampling layer 3
Y15 = upSampling2D(Y14, 256, 256)

# Define the loss function 
loss = tf.reduce_mean(tf.squared_difference(Y15, Y_)) 

# Define an optimizer
train_step = tf.train.AdamOptimizer(0.005).minimize(loss)

In [26]:
def training_step(i, batch_X, batch_Y):
    print batch_X
    print batch_Y
    print "\r", i,
    sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y})

In [27]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  
    # initialize the variables
    sess.run(init)

    # initialize the queue threads to start to shovel data
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print "from the train set:"
    images, labels = iterator.get_next()
    for i in range(12):
        print sess.run(training_step(i, images.eval(), labels.eval()))

    # stop our queue threads and properly close the session
    coord.request_stop()
    coord.join(threads)
    sess.close()

from the train set:
[[[[ 0.2073524 ]
   [ 0.20888492]
   [ 0.2110353 ]
   ..., 
   [ 0.19899313]
   [ 0.19968859]
   [ 0.19822832]]

  [[ 0.20958038]
   [ 0.21119584]
   [ 0.2110353 ]
   ..., 
   [ 0.19984501]
   [ 0.19914832]
   [ 0.19807313]]

  [[ 0.20888492]
   [ 0.21088011]
   [ 0.21088011]
   ..., 
   [ 0.2000002 ]
   [ 0.20183897]
   [ 0.19883671]]

  ..., 
  [[ 0.06360747]
   [ 0.06445936]
   [ 0.06569096]
   ..., 
   [ 0.06698824]
   [ 0.06622466]
   [ 0.06622466]]

  [[ 0.06384561]
   [ 0.06277042]
   [ 0.06606946]
   ..., 
   [ 0.07052542]
   [ 0.06622466]
   [ 0.06407427]]

  [[ 0.06714465]
   [ 0.06830156]
   [ 0.06499304]
   ..., 
   [ 0.06729985]
   [ 0.06453572]
   [ 0.06407427]]]]
[[[[-0.01100015  0.00786799]
   [-0.00391246 -0.01142263]
   [-0.01340233 -0.00135665]
   ..., 
   [-0.00616937 -0.00213678]
   [-0.00616924 -0.00213704]
   [-0.00616883 -0.00213782]]

  [[-0.0065713  -0.00733658]
   [-0.01824293  0.00866676]
   [-0.01462229  0.0082688 ]
   ..., 
   [-0.00834

TypeError: Fetch argument None has invalid type <type 'NoneType'>